# Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import requests
from requests import get
from os import path
from bs4 import BeautifulSoup
import os
import re

import json
import unicodedata

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

from typing import Dict, List, Optional, Union, cast
from env import github_token, github_username

import acquire as ac
import prepare

# Acquire

#### For this project, you will have to build a dataset yourself. Decide on a list of GitHub repositories to scrape, and write the python code necessary to extract the text of the README file for each page, and the primary language of the repository.

#### Which repositories you use are up to you, but you should include at least 100 repositories in your data set.

In [2]:
url = 'https://github.com/search?o=desc&p={num}&q=OpenCV&s=stars&type=Repositories'
response = requests.get(url)

In [3]:
urls = ac.get_all_urls() 

In [4]:
urls

['oarriaga/face_classification',
 'spmallick/learnopencv',
 'opencv/opencv',
 'opencv/opencv_contrib',
 'vipstone/faceai',
 'Hironsan/BossSensor',
 'CMU-Perceptual-Computing-Lab/openpose',
 'PySimpleGUI/PySimpleGUI',
 'Ewenwan/MVision',
 'openframeworks/openFrameworks',
 'jrosebr1/imutils',
 'bijection/sistine',
 'peterbraden/node-opencv',
 'kelaberetiv/TagUI',
 'esimov/pigo',
 'nuno-faria/tiler',
 'hybridgroup/gocv',
 'justadudewhohacks/opencv4nodejs',
 'bytedeco/javacv',
 'hamuchiwa/AutoRCCar',
 'MasteringOpenCV/code',
 'CodecWang/OpenCV-Python-Tutorial',
 'anandpawara/Real_Time_Image_Animation',
 'mapillary/OpenSfM',
 'shimat/opencvsharp',
 'HuTianQi/SmartOpenCV',
 'Roujack/mathAI',
 'soruly/trace.moe',
 'amusi/AI-Job-Notes',
 'makelove/OpenCV-Python-Tutorial',
 'oreillymedia/Learning-OpenCV-3_examples',
 'kongqw/OpenCVForAndroid',
 'nagadomi/lbpcascade_animeface',
 'tebelorg/RPA-Python',
 'abhiTronix/vidgear',
 'changwookjun/StudyBook',
 'ivanseidel/Is-Now-Illegal',
 'andrewssobral

In [5]:
data = pd.read_json('data.json')

In [6]:
# 150 repos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             150 non-null    object
 1   language         144 non-null    object
 2   readme_contents  150 non-null    object
dtypes: object(3)
memory usage: 3.6+ KB


In [7]:
data

,repo,language,readme_contents
0,Ewenwan/MVision,C++,# MVision Machine Vision 机器视觉\n[AI算法工程师手册 数学基础...
1,opencv/opencv_contrib,C++,## Repository for OpenCV's extra modules\n\nTh...
2,PySimpleGUI/PySimpleGUI,Python,"\n<p align=""center"">\n <img src=""https://raw...."
3,opencv/opencv,C++,## OpenCV: Open Source Computer Vision Library...
4,CMU-Perceptual-Computing-Lab/openpose,C++,"<div align=""center"">\n <img src="".github/Lo..."
...,...,...,...
145,jerry1900/faceRecognition,Python,# faceRecognition\n利用OpenCV、CNN进行人脸识别\n万壑，4978...
146,lixiaoshaxing/MultiMediaLearn,C,# MultiMediaLearn\n多媒体学习：图片处理，音视频处理，相机使用，OpenG...
147,WL-Amigo/waifu2x-converter-cpp,C++,# waifu2x (converter only version)\n\nThis is ...
148,atulapra/Emotion-detection,Python,# Emotion detection using deep learning\n\n## ...


# Prepare

# Explore

#### Explore the data that you have scraped. Here are some ideas for exploration:

- What are the most common words in READMEs?   


- What does the distribution of IDFs look like for the most common words?  


- Does the length of the README vary by programming language?    


- Do different programming languages use a different number of unique words?

# Model

- Transform your documents into a form that can be used in a machine learning model. You should use the programming language of the repository as the label to predict.  


- Try fitting several different models and using several different representations of the text (e.g. a simple bag of words, then also the TF-IDF values for each).   


- Build a function that will take in the text of a README file, and tries to predict the programming language.  